In [22]:
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import os 
import gc

import torch
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim


from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score


In [ ]:
gList = ['/home/jua/corsika/run/energy_var/npz/square/0514/gamma_square_n51.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_0.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_1.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_2.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_3.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_4.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_5.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_6.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_7.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_8.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0513/gamma_square_n51_9.npz'
            ]

In [29]:
pList = ['/home/jua/corsika/run/energy_var/npz/square/0513/proton_square_n51.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_0.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_1.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_2.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_3.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_4.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_5.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_6.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_7.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_8.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0514/proton_square_n51_9.npz'
            ]

In [23]:
'''
pList = [
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_0.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_1.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_2.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_3.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_4.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_5.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_6.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_7.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_8.npz',
             '/home/jua/corsika/run/energy_var/npz/square/0520/proton_square_n51_9.npz'
            ]
'''

In [24]:
perPMTHit = [6.7,10.5,16.2,24.7,35.6,48.5,61.8,74.0,85.0,100.0]

In [25]:
# bin all
def SeparateBinHit(image, energy, binHit):
    n = len(image)
    for imu, iml, im, en in zip(image[:,0][:n],image[:,3][:n], image[:n], energy[:n]):
        percent = (len(imu[imu!=0])+len(iml[iml!=0]))*100/(51.0*51.0*2)
        for i in range(9):
            if perPMTHit[i] < percent < perPMTHit[i+1]:
                binHit[i][0].append(im)
                binHit[i][1].append(en)

In [30]:
for n,file in enumerate(pList,10):
    pbinHitAll = [[[],[]] for i in range(9)]
    proton = np.load(file)
    pim = proton['arr_0']
    pen = proton['arr_1'] 
    SeparateBinHit(pim, pen, pbinHitAll)
    
    for i in range(9):
        pbinHitAll[i][0] = np.asarray(pbinHitAll[i][0])
        pbinHitAll[i][1] = np.asarray(pbinHitAll[i][1])
        np.savez_compressed('/home/jua/corsika/run/energy_var/npz/square/0522/bin/proton_energy_bin{}_{}.npz'.format(i+1,n),
                        image=np.asarray(pbinHitAll[i][0]),energy=np.asarray(pbinHitAll[i][1]))
    del proton, pim, pen, pbinHitAll
    gc.collect()    
    #pbinHitAll = [[[],[]] for i in range(9)]

In [37]:
for n,file in enumerate(gList):
    gbinHitAll = [[[],[]] for i in range(9)]
    gamma = np.load(file)
    gim = gamma['arr_0']
    gen = gamma['arr_1'] 
    SeparateBinHit(gim, gen, gbinHitAll)
    
    for i in range(9):
        gbinHitAll[i][0] = np.asarray(gbinHitAll[i][0])
        gbinHitAll[i][1] = np.asarray(gbinHitAll[i][1])
        np.savez_compressed('/home/jua/corsika/run/energy_var/npz/square/0522/bin/gamma_energy_bin{}_{}.npz'.format(i+1,n),
                        image=np.asarray(gbinHitAll[i][0]),energy=np.asarray(gbinHitAll[i][1]))
    del gamma, gim, gen, gbinHitAll
    gc.collect() 

In [35]:
del gamma, gim, gen, gbinHitAll

In [36]:
gc.collect()

8